In [1]:
##DOWNLOAD THE REQUIRED MODULES
import sys
!conda install --yes --prefix {sys.prefix} numpy

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\asus\Anaconda3

  added / updated specs:
    - numpy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.1                |   py38haa95532_0         2.9 MB
    ------------------------------------------------------------
                                           Total:         2.9 MB

The following packages will be UPDATED:

  conda                                        4.9.0-py38_0 --> 4.9.1-py38haa95532_0




conda-4.9.1          | 2.9 MB    |            |   0% 
conda-4.9.1          | 2.9 MB    |            |   1% 
conda-4.9.1          | 2.9 MB    | 2          |   2% 
conda-4.9.1          | 2.9 MB    | 3          |   4% 
conda-4.9.1          | 2.9 MB    | 8          |   9% 
conda-4.9.1          | 2.9 MB    | ##1        |  21% 
conda-4.9.1          | 2.9 MB    | ##8        |  29% 
con

In [2]:
!conda install --yes --prefix {sys.prefix} opencv

Solving environment: ...working... done

# All requested packages already installed.



In [ ]:
import numpy as np
import cv2
import math

capt = cv2.VideoCapture(0)

while capt.isOpened():

    ret, frame = capt.read()

    cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 0)
    crop_im = frame[100:300, 100:300]

    blur = cv2.GaussianBlur(crop_im, (3, 3), 0)

    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv, np.array([2, 0, 0]), np.array([20, 255, 255]))

    kernel = np.ones((5, 5))

    dilation = cv2.dilate(mask, kernel, iterations=1)
    erosion = cv2.erode(dilation, kernel, iterations=1)

    filtered = cv2.GaussianBlur(erosion, (3, 3), 0)
    ret, thresh = cv2.threshold(filtered, 127, 255, 0)

    cv2.imshow("Thresholded", thresh)

    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    try:
        contour = max(contours, key=lambda x: cv2.contourArea(x))

        u, v, w, h = cv2.boundingRect(contour)
        cv2.rectangle(crop_im, (u, v), (u + w, v + h), (0, 0, 255), 0)

        hull = cv2.convexHull(contour)

        drawing = np.zeros(crop_im.shape, np.uint8)
        cv2.drawContours(drawing, [contour], -1, (0, 255, 0), 0)
        cv2.drawContours(drawing, [hull], -1, (0, 0, 255), 0)

        hull = cv2.convexHull(contour, returnPoints=False)
        defects = cv2.convexityDefects(contour, hull)

        count_defects = 0

        for i in range(defects.shape[0]):
            p, e, f, d = defects[i, 0]
            start = tuple(contour[p][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])

            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14

            if angle <= 90:
                count_defects += 1
                cv2.circle(crop_im, far, 1, [0, 0, 255], -1)

            cv2.line(crop_im, start, end, [0, 255, 0], 2)
        #engine=pyttsx3.init('sapi5')

        if count_defects == 0:
            cv2.putText(frame, "Alright", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255),2)
          
            
        if count_defects == 1:
            cv2.putText(frame, "Peace", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
           
        if count_defects == 2:
            cv2.putText(frame, "I am not well", (5, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
          
        if count_defects == 3:
            cv2.putText(frame, "I need water", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
           
        if count_defects == 4:
            cv2.putText(frame, "Hello", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2,(0,0,255), 2)
            
        else:
            pass
    except:
        pass

    cv2.imshow("Gesture", frame)
    all_image = np.hstack((drawing, crop_im))
    cv2.imshow('Contours', all_image)

    if cv2.waitKey(1) == ord('q'):
        break

capt.release()
cv2.destroyAllWindows()
